In [ ]:
import torch

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Torch CUDA:", torch.version.cuda)
print("GPU:", torch.cuda.get_device_name(0))

In [ ]:


import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
import matplotlib.pyplot as plt

# ==========================================
# 0) HELPERS
# ==========================================

def pct_to_count(pct, total, min_count=1):
    """pct in [0,1] -> integer count with minimum count."""
    return max(int(min_count), int(round(float(pct) * int(total))))

def clamp_query(n_query, n_unlabeled):
    """Ensure we don't query more than available unlabeled points."""
    return int(min(int(n_query), int(n_unlabeled)))

def random_undersample(X, y, seed=1):
    """
    Random under-sampling to the minority class size.
    Works for binary labels {0,1}. For non-binary/multi-class, it returns unchanged.
    """
    y = np.asarray(y)
    classes, counts = np.unique(y, return_counts=True)
    if len(classes) != 2:
        return X, y

    rng = np.random.RandomState(seed)
    c0, c1 = classes[0], classes[1]
    n0, n1 = counts[0], counts[1]
    target = min(n0, n1)

    idx0 = np.where(y == c0)[0]
    idx1 = np.where(y == c1)[0]

    sel0 = rng.choice(idx0, size=target, replace=False)
    sel1 = rng.choice(idx1, size=target, replace=False)

    sel = np.concatenate([sel0, sel1])
    rng.shuffle(sel)

    return X[sel], y[sel]

# ==========================================
# 1) DATASET & ACTIVE DATA
# ==========================================

class Tabular_Handler(Dataset):
    def __init__(self, X, Y):
        self.X = X.float() if isinstance(X, torch.Tensor) else torch.tensor(X, dtype=torch.float32)
        self.Y = Y.long() if isinstance(Y, torch.Tensor) else torch.tensor(Y, dtype=torch.long)

    def __getitem__(self, index):
        return self.X[index], self.Y[index], index

    def __len__(self):
        return len(self.X)

class ActiveData:
    def __init__(self, X_train, Y_train, X_test, Y_test, handler):
        self.X_train, self.Y_train = X_train, Y_train
        self.X_test, self.Y_test = X_test, Y_test
        self.handler = handler
        self.n_pool = len(X_train)
        self.labeled_idxs = np.zeros(self.n_pool, dtype=bool)

    def initialize_labels(self, num):
        num = int(min(max(0, num), self.n_pool))
        tmp_idxs = np.arange(self.n_pool)
        np.random.shuffle(tmp_idxs)
        self.labeled_idxs[tmp_idxs[:num]] = True

    def get_labeled_data(self):
        idxs = np.where(self.labeled_idxs)[0]
        return idxs, self.handler(self.X_train[self.labeled_idxs], self.Y_train[self.labeled_idxs])

    def get_unlabeled_data(self):
        idxs = np.where(~self.labeled_idxs)[0]
        return idxs, self.handler(self.X_train[~self.labeled_idxs], self.Y_train[~self.labeled_idxs])

    def get_test_data(self):
        return self.handler(self.X_test, self.Y_test)

    def get_train_data(self):
        # For KCenterGreedy: needs all train data + current labeled mask
        return self.labeled_idxs.copy(), self.handler(self.X_train, self.Y_train)

# ==========================================
# 2) NETWORK (WITH DROPOUT)
# ==========================================

class TabularNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, n_classes=2, dropout_p=0.5):
        super(TabularNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, n_classes)
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h1 = self.dropout(h1)
        h2 = F.relu(self.fc2(h1))
        h2 = self.dropout(h2)
        logits = self.fc3(h2)
        return logits, h2  # logits, embedding

# ==========================================
# 3) BASE STRATEGY (STANDARD + MC-DROPOUT + EMBEDDINGS)
# ==========================================

class Strategy:
    def __init__(self, dataset, net_builder, device):
        self.dataset = dataset
        self.net_builder = net_builder
        self.device = device
        self.clf = None
        self.n_classes = None

    def update(self, idxs):
        self.dataset.labeled_idxs[idxs] = True

    def train(self, n_epochs=15, lr=0.001, batch_size=64):
        _, data = self.dataset.get_labeled_data()
        self.clf = self.net_builder().to(self.device)
        self.n_classes = self.clf.fc3.out_features

        optimizer = optim.Adam(self.clf.parameters(), lr=lr)
        loader = DataLoader(data, batch_size=batch_size, shuffle=True)

        self.clf.train()
        for _ in range(n_epochs):
            for x, y, _ in loader:
                x, y = x.to(self.device), y.to(self.device)
                optimizer.zero_grad()
                out, _ = self.clf(x)
                loss = F.cross_entropy(out, y)
                loss.backward()
                optimizer.step()

    def predict_prob(self, data):
        self.clf.eval()
        loader = DataLoader(data, batch_size=1000, shuffle=False)
        probs = torch.zeros([len(data), self.n_classes])

        with torch.no_grad():
            for x, _, idxs in loader:
                out, _ = self.clf(x.to(self.device))
                probs[idxs] = F.softmax(out, dim=1).cpu()
        return probs

    def predict_prob_dropout(self, data, n_drop=10):
        prev_mode = self.clf.training
        self.clf.train()

        n = len(data)
        probs_sum = torch.zeros([n, self.n_classes])

        with torch.no_grad():
            for _ in range(n_drop):
                loader = DataLoader(data, batch_size=1000, shuffle=False)
                for x, _, idxs in loader:
                    out, _ = self.clf(x.to(self.device))
                    probs_sum[idxs] += F.softmax(out, dim=1).cpu()

        self.clf.train(prev_mode)
        return probs_sum / n_drop

    def predict_prob_dropout_split(self, data, n_drop=10):
        prev_mode = self.clf.training
        self.clf.train()

        n = len(data)
        probs_all = torch.zeros([n_drop, n, self.n_classes])

        with torch.no_grad():
            for k in range(n_drop):
                loader = DataLoader(data, batch_size=1000, shuffle=False)
                for x, _, idxs in loader:
                    out, _ = self.clf(x.to(self.device))
                    probs_all[k, idxs] = F.softmax(out, dim=1).cpu()

        self.clf.train(prev_mode)
        return probs_all

    def get_embeddings(self, data):
        self.clf.eval()
        loader = DataLoader(data, batch_size=1000, shuffle=False)

        with torch.no_grad():
            for x, _, _ in loader:
                _, emb = self.clf(x.to(self.device))
                emb_dim = emb.shape[1]
                break

        embeddings = torch.zeros(len(data), emb_dim)
        with torch.no_grad():
            loader = DataLoader(data, batch_size=1000, shuffle=False)
            for x, _, idxs in loader:
                _, emb = self.clf(x.to(self.device))
                embeddings[idxs] = emb.cpu()
        return embeddings

# ==========================================
# 4) QUERY STRATEGIES (INCLUDING "Full")
# ==========================================

class Full(Strategy):
    """No active learning: use all training data once."""
    def query(self, n):
        return np.array([], dtype=int)

class RandomSampling(Strategy):
    def query(self, n):
        idxs = np.where(~self.dataset.labeled_idxs)[0]
        return np.random.choice(idxs, n, replace=False)

class EntropySampling(Strategy):
    def query(self, n):
        idxs, data = self.dataset.get_unlabeled_data()
        probs = self.predict_prob(data)
        ent = -(probs * torch.log(probs + 1e-10)).sum(1)
        return idxs[ent.sort(descending=True)[1][:n]]

class LeastConfidence(Strategy):
    def query(self, n):
        idxs, data = self.dataset.get_unlabeled_data()
        probs = self.predict_prob(data)
        max_probs = probs.max(1)[0]
        return idxs[max_probs.sort()[1][:n]]

class MarginSampling(Strategy):
    def query(self, n):
        idxs, data = self.dataset.get_unlabeled_data()
        probs = self.predict_prob(data)
        probs_sorted, _ = probs.sort(descending=True)
        margins = probs_sorted[:, 0] - probs_sorted[:, 1]
        return idxs[margins.sort()[1][:n]]

class KMeansSampling(Strategy):
    def query(self, n):
        unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
        emb = self.get_embeddings(unlabeled_data).numpy()

        if emb.shape[0] <= n:
            return unlabeled_idxs

        n_clusters = min(n, emb.shape[0])
        km = KMeans(n_clusters=n_clusters, n_init="auto", random_state=0)
        km.fit(emb)

        cluster_idxs = km.predict(emb)
        centers = km.cluster_centers_[cluster_idxs]
        dis = ((emb - centers) ** 2).sum(axis=1)

        selected = []
        used = np.zeros(emb.shape[0], dtype=bool)

        for c in np.unique(cluster_idxs):
            members = np.where(cluster_idxs == c)[0]
            if members.size == 0:
                continue
            chosen = members[np.argmin(dis[members])]
            selected.append(chosen)
            used[chosen] = True

        selected = np.array(selected, dtype=int)
        if selected.size < n:
            remaining = np.where(~used)[0]
            remaining = remaining[np.argsort(dis[remaining])]
            extra = remaining[: (n - selected.size)]
            q = np.concatenate([selected, extra])
        else:
            q = selected[:n]

        return unlabeled_idxs[q]

class KCenterGreedy(Strategy):
    def query(self, n):
        labeled_mask, train_data = self.dataset.get_train_data()
        emb = self.get_embeddings(train_data).numpy()

        dist_mat = np.matmul(emb, emb.T)
        sq = np.array(dist_mat.diagonal()).reshape(len(labeled_mask), 1)
        dist_mat *= -2
        dist_mat += sq
        dist_mat += sq.T
        dist_mat = np.sqrt(np.maximum(dist_mat, 0.0))

        mat = dist_mat[~labeled_mask, :][:, labeled_mask]

        for _ in tqdm(range(n), ncols=100, desc="KCenterGreedy"):
            mat_min = mat.min(axis=1)
            q_idx_ = mat_min.argmax()
            q_idx = np.arange(self.dataset.n_pool)[~labeled_mask][q_idx_]
            labeled_mask[q_idx] = True
            mat = np.delete(mat, q_idx_, 0)
            mat = np.append(mat, dist_mat[~labeled_mask, q_idx][:, None], axis=1)

        return np.arange(self.dataset.n_pool)[(self.dataset.labeled_idxs ^ labeled_mask)]

class BALDDropout(Strategy):
    def __init__(self, dataset, net_builder, device, n_drop=10):
        super().__init__(dataset, net_builder, device)
        self.n_drop = n_drop

    def query(self, n):
        unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
        probs = self.predict_prob_dropout_split(unlabeled_data, n_drop=self.n_drop)
        pb = probs.mean(0)
        entropy1 = (-pb * torch.log(pb + 1e-10)).sum(1)
        entropy2 = (-probs * torch.log(probs + 1e-10)).sum(2).mean(0)
        bald = entropy2 - entropy1
        return unlabeled_idxs[bald.sort(descending=True)[1][:n]]

class EntropySamplingDropout(Strategy):
    def __init__(self, dataset, net_builder, device, n_drop=10):
        super().__init__(dataset, net_builder, device)
        self.n_drop = n_drop

    def query(self, n):
        unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
        probs = self.predict_prob_dropout(unlabeled_data, n_drop=self.n_drop)
        ent = -(probs * torch.log(probs + 1e-10)).sum(1)
        return unlabeled_idxs[ent.sort(descending=True)[1][:n]]

class LeastConfidenceDropout(Strategy):
    def __init__(self, dataset, net_builder, device, n_drop=10):
        super().__init__(dataset, net_builder, device)
        self.n_drop = n_drop

    def query(self, n):
        unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
        probs = self.predict_prob_dropout(unlabeled_data, n_drop=self.n_drop)
        max_probs = probs.max(1)[0]
        return unlabeled_idxs[max_probs.sort()[1][:n]]

class MarginSamplingDropout(Strategy):
    def __init__(self, dataset, net_builder, device, n_drop=10):
        super().__init__(dataset, net_builder, device)
        self.n_drop = n_drop

    def query(self, n):
        unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
        probs = self.predict_prob_dropout(unlabeled_data, n_drop=self.n_drop)
        probs_sorted, _ = probs.sort(descending=True)
        margins = probs_sorted[:, 0] - probs_sorted[:, 1]
        return unlabeled_idxs[margins.sort()[1][:n]]

class AdversarialBIM(Strategy):
    def __init__(self, dataset, net_builder, device, eps=0.05, max_iter=20):
        super().__init__(dataset, net_builder, device)
        self.eps = eps
        self.max_iter = max_iter

    def cal_dis(self, x):
        self.clf.eval()
        nx = x.unsqueeze(0).to(self.device)
        nx_orig = nx.clone()
        eta = torch.zeros_like(nx, device=self.device, requires_grad=True)

        with torch.no_grad():
            out, _ = self.clf(nx_orig)
            py = out.max(1)[1].item()

        ny = py
        it = 0
        while ny == py and it < self.max_iter:
            self.clf.zero_grad()
            out, _ = self.clf(nx_orig + eta)
            loss = F.cross_entropy(out, torch.tensor([py], device=self.device))
            loss.backward()
            with torch.no_grad():
                eta += self.eps * eta.grad.sign()
                eta.grad.zero_()
                out2, _ = self.clf(nx_orig + eta)
                ny = out2.max(1)[1].item()
            it += 1

        return (eta.detach() ** 2).sum().item()

    def query(self, n):
        unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
        dis = np.zeros(unlabeled_idxs.shape, dtype=np.float32)
        for i in tqdm(range(len(unlabeled_idxs)), ncols=100, desc="AdversarialBIM"):
            x, _, _ = unlabeled_data[i]
            dis[i] = self.cal_dis(x)
        return unlabeled_idxs[dis.argsort()[:n]]

class AdversarialDeepFool(Strategy):
    def __init__(self, dataset, net_builder, device, max_iter=50):
        super().__init__(dataset, net_builder, device)
        self.max_iter = max_iter

    def cal_dis(self, x):
        self.clf.eval()
        nx = x.unsqueeze(0).to(self.device)
        nx.requires_grad_()
        eta = torch.zeros_like(nx, device=self.device)

        with torch.no_grad():
            out, _ = self.clf(nx)
            num_classes = out.shape[1]
            py = out.max(1)[1].item()

        ny = py
        it = 0
        while ny == py and it < self.max_iter:
            self.clf.zero_grad()
            out, _ = self.clf(nx + eta)
            out[0, py].backward(retain_graph=True)
            grad_orig = nx.grad.data.clone()
            nx.grad.data.zero_()

            best_val = np.inf
            ri = None

            for k in range(num_classes):
                if k == py:
                    continue
                self.clf.zero_grad()
                out, _ = self.clf(nx + eta)
                out[0, k].backward(retain_graph=True)
                grad_k = nx.grad.data.clone()
                nx.grad.data.zero_()

                wi = grad_k - grad_orig
                fi = (out[0, k] - out[0, py]).item()

                wi_np = wi.detach().cpu().numpy().ravel()
                wi_norm = np.linalg.norm(wi_np) + 1e-12
                val = abs(fi) / wi_norm

                if val < best_val:
                    ri = (val / wi_norm) * wi
                    best_val = val

            if ri is None:
                break

            with torch.no_grad():
                eta += ri
                out2, _ = self.clf(nx + eta)
                ny = out2.max(1)[1].item()

            it += 1

        return (eta.detach() ** 2).sum().item()

    def query(self, n):
        unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
        dis = np.zeros(unlabeled_idxs.shape, dtype=np.float32)
        for i in tqdm(range(len(unlabeled_idxs)), ncols=100, desc="AdversarialDeepFool"):
            x, _, _ = unlabeled_data[i]
            dis[i] = self.cal_dis(x)
        return unlabeled_idxs[dis.argsort()[:n]]

# ==========================================
# 5) MAIN EXPERIMENT (WITH OPTIONAL UNDER-SAMPLING + FULL BASELINE)
# ==========================================

def run_experiment(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    if not os.path.exists(args.csv_path):
        print(f"Error: {args.csv_path} not found.")
        return

    df = pd.read_csv(args.csv_path)

    X = df.drop(columns=[args.label_col]).values
    y = df[args.label_col].values
    if y.dtype != np.int64 and y.dtype != np.int32:
        y = LabelEncoder().fit_transform(y)

    # -------- Optional under-sampling BEFORE split --------
    if args.use_undersampling and args.undersampling_stage == "before_split":
        X, y = random_undersample(X, y, seed=args.seed)
        print(f"[Under-sampling BEFORE split] New distribution: {np.bincount(y)} | N={len(y)}")

    # Scale after optional pre-split under-sampling
    X = StandardScaler().fit_transform(X)
    input_dim = X.shape[1]

    # Split
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=args.seed)

    # -------- Optional under-sampling AFTER split --------
    if args.use_undersampling and args.undersampling_stage == "after_split":
        X_tr, y_tr = random_undersample(X_tr, y_tr, seed=args.seed)
        print(f"[Under-sampling AFTER split] Train distribution: {np.bincount(y_tr)} | N_train={len(y_tr)}")
        print(f"[Under-sampling AFTER split] Test  distribution (unchanged): {np.bincount(y_te)} | N_test={len(y_te)}")

    n_pool = len(X_tr)
    n_classes = len(np.unique(y))  # keep global number of classes

    print(f"Train/test split: {len(X_tr)}/{len(X_te)} | n_pool={n_pool} | n_classes={n_classes}")
    print(f"Overall class distribution (current y): {np.bincount(y)}")

    strategy_map = {
        "Full": Full,
        "RandomSampling": RandomSampling,
        "EntropySampling": EntropySampling,
        "LeastConfidence": LeastConfidence,
        "MarginSampling": MarginSampling,
        "KMeansSampling": KMeansSampling,
        "KCenterGreedy": KCenterGreedy,
        "BALDDropout": BALDDropout,
        "EntropySamplingDropout": EntropySamplingDropout,
        "LeastConfidenceDropout": LeastConfidenceDropout,
        "MarginSamplingDropout": MarginSamplingDropout,
        "AdversarialBIM": AdversarialBIM,
        "AdversarialDeepFool": AdversarialDeepFool,
    }

    os.makedirs(args.results_dir, exist_ok=True)
    dataset_name = os.path.splitext(os.path.basename(args.csv_path))[0]

    all_curves = {}   # name -> (x_pct, acc)
    results_list = []

    for strat_name in args.strategies:
        if strat_name not in strategy_map:
            print(f"Strategy {strat_name} not found, skipping.")
            continue

        print(f"\n=== Strategy: {strat_name} ===")

        dataset = ActiveData(X_tr, y_tr, X_te, y_te, Tabular_Handler)

        # ---- Full baseline: label everything ----
        if strat_name == "Full":
            dataset.labeled_idxs[:] = True
            n_init = int(dataset.labeled_idxs.sum())
            print(f"[Full] Using all training data: {n_init} ({(n_init/n_pool)*100:.2f}%)")
        else:
            n_init = min(pct_to_count(args.init_pct, n_pool, min_count=args.min_init), n_pool)
            dataset.initialize_labels(n_init)
            print(f"Initial labeled: {n_init} ({(n_init/n_pool)*100:.2f}% of train pool)")

        net_builder = lambda: TabularNet(
            input_dim=input_dim,
            hidden_dim=args.hidden_dim,
            n_classes=n_classes,
            dropout_p=args.dropout_p
        )

        StratClass = strategy_map[strat_name]
        if strat_name in ["BALDDropout", "EntropySamplingDropout", "LeastConfidenceDropout", "MarginSamplingDropout"]:
            strategy = StratClass(dataset, net_builder, device, n_drop=args.n_drop)
        elif strat_name == "AdversarialBIM":
            strategy = StratClass(dataset, net_builder, device, eps=args.bim_eps, max_iter=args.bim_iter)
        elif strat_name == "AdversarialDeepFool":
            strategy = StratClass(dataset, net_builder, device, max_iter=args.deepfool_iter)
        else:
            strategy = StratClass(dataset, net_builder, device)

        x_pct, y_acc = [], []

        best_metric = None
        bad_rounds = 0

        # Full runs once; AL runs up to args.n_round
        n_rounds_effective = 0 if strat_name == "Full" else args.n_round

        for rd in range(n_rounds_effective + 1):
            # --------------------------
            # 1) TRAIN + EVAL TIME
            # --------------------------
            train_eval_start = time.time()

            strategy.train(n_epochs=args.n_epochs, lr=args.lr, batch_size=args.batch_size)

            loader = DataLoader(dataset.get_test_data(), batch_size=1000, shuffle=False)
            y_true, y_pred = [], []

            strategy.clf.eval()
            with torch.no_grad():
                for xb, yb, _ in loader:
                    logits, _ = strategy.clf(xb.to(device))
                    preds = logits.argmax(1).cpu().numpy()
                    y_pred.extend(preds)
                    y_true.extend(yb.cpu().numpy())

            train_eval_time = time.time() - train_eval_start

            acc = accuracy_score(y_true, y_pred)

            if len(np.unique(y_true)) == 2:
                tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
                prec = precision_score(y_true, y_pred, average="binary", zero_division=0)
                rec = recall_score(y_true, y_pred, average="binary", zero_division=0)
                f1 = f1_score(y_true, y_pred, average="binary", zero_division=0)
            else:
                tn = fp = fn = tp = 0
                prec = precision_score(y_true, y_pred, average="macro", zero_division=0)
                rec = recall_score(y_true, y_pred, average="macro", zero_division=0)
                f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)

            n_labeled_now = int(dataset.labeled_idxs.sum())
            pct_labeled_now = (n_labeled_now / n_pool) * 100.0

            # Store curve points
            x_pct.append(pct_labeled_now)
            y_acc.append(acc)

            print(
                f" Round {rd}: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}, "
                f"Labeled={n_labeled_now} ({pct_labeled_now:.2f}%), Train+Eval={train_eval_time:.2f}s"
            )

            # Prepare row (we will fill query_time_sec AFTER querying)
            row = {
                "dataset": dataset_name,
                "strategy": strat_name,
                "round": rd,
                "n_labeled": n_labeled_now,
                "pct_labeled": pct_labeled_now,
                "accuracy": acc,
                "precision": prec,
                "recall": rec,
                "f1": f1,
                "tn": int(tn),
                "fp": int(fp),
                "fn": int(fn),
                "tp": int(tp),
                "train_eval_time_sec": float(train_eval_time),
                "query_time_sec": 0.0,
                "total_time_sec": float(train_eval_time),
                "stop_reason": "",
                "undersampling": bool(args.use_undersampling),
                "undersampling_stage": str(args.undersampling_stage),
            }
            results_list.append(row)

            # Full baseline: no querying, no stopping logic needed
            if strat_name == "Full":
                results_list[-1]["stop_reason"] = "full_training(no_active_learning)"
                break

            # --------------------------
            # 2) CONVERGENCE CHECKS
            # --------------------------
            current_metric = f1 if args.metric_name.lower() == "f1" else acc

            if best_metric is None:
                best_metric = current_metric
                bad_rounds = 0
            else:
                if current_metric > best_metric + args.min_delta:
                    best_metric = current_metric
                    bad_rounds = 0
                else:
                    bad_rounds += 1

            if pct_labeled_now >= args.max_label_pct:
                results_list[-1]["stop_reason"] = f"max_label_pct_reached({args.max_label_pct}%)"
                print(f"Stopping: reached max_label_pct={args.max_label_pct}% (now {pct_labeled_now:.2f}%)")
                break

            if rd >= args.min_rounds_before_stop and bad_rounds >= args.patience:
                results_list[-1]["stop_reason"] = (
                    f"plateau(patience={args.patience}, min_delta={args.min_delta}, metric={args.metric_name})"
                )
                print(f"Stopping: no improvement > {args.min_delta} for {args.patience} consecutive rounds.")
                break

            # --------------------------
            # 3) QUERY TIME
            # --------------------------
            if rd < n_rounds_effective:
                n_unlabeled = int((~dataset.labeled_idxs).sum())
                if n_unlabeled <= 0:
                    results_list[-1]["stop_reason"] = "no_unlabeled_remaining"
                    print("No unlabeled points remaining. Stopping early.")
                    break

                n_q = pct_to_count(args.query_pct, n_pool, min_count=args.min_query)
                n_q = clamp_query(n_q, n_unlabeled)
                if n_q <= 0:
                    results_list[-1]["stop_reason"] = "query_size_zero_after_clamp"
                    print("Query size became 0 after clamping. Stopping early.")
                    break

                query_start = time.time()
                q_idxs = strategy.query(n_q)
                query_time = time.time() - query_start

                strategy.update(q_idxs)

                results_list[-1]["query_time_sec"] = float(query_time)
                results_list[-1]["total_time_sec"] = float(train_eval_time + query_time)

                print(f"   Query time: {query_time:.2f}s | Total: {train_eval_time+query_time:.2f}s")

        all_curves[strat_name] = (x_pct, y_acc)

    # Save metrics CSV (one file per dataset)
    results_df = pd.DataFrame(results_list)
    out_csv = os.path.join(args.results_dir, "metrics_results_" + dataset_name + ".csv")
    results_df.to_csv(out_csv, index=False)
    print(f"\nSaved detailed results to: {out_csv}")

    # -------- Plotting configs --------
    label_map = {
        "Full": "Full (100%)",
        "RandomSampling": "Random",
        "EntropySampling": "Entropy",
        "LeastConfidence": "Least Confidence",
        "MarginSampling": "Margin",
        "KMeansSampling": "K-Means",
        "KCenterGreedy": "K-Center",
        "BALDDropout": "BALD (MC-Dropout)",
        "EntropySamplingDropout": "Entropy (MC-Dropout)",
        "LeastConfidenceDropout": "Least Confidence (MC-Dropout)",
        "MarginSamplingDropout": "Margin (MC-Dropout)",
        "AdversarialBIM": "BIM",
        "AdversarialDeepFool": "DeepFool",
    }

    marker_map = {
        "Full": "8",
        "RandomSampling": "o",
        "EntropySampling": "s",
        "LeastConfidence": "^",
        "MarginSampling": "D",
        "KMeansSampling": "P",
        "KCenterGreedy": "X",
        "BALDDropout": "*",
        "EntropySamplingDropout": "v",
        "LeastConfidenceDropout": "<",
        "MarginSamplingDropout": ">",
        "AdversarialBIM": "h",
        "AdversarialDeepFool": "p",
    }

    # Plot learning curves (% labeled on x-axis)
    if len(all_curves) > 0:
        plt.figure(figsize=(10, 6))

        for name, (xp, ya) in all_curves.items():
            display_name = label_map.get(name, name)
            marker = marker_map.get(name, "o")
            plt.plot(xp, ya, label=display_name, marker=marker)

        plt.xlabel("% of training pool labeled")
        plt.ylabel("Accuracy")
        plt.title(dataset_name)
        plt.grid(True)

        # Legend below
        plt.legend(
            loc="upper center",
            bbox_to_anchor=(0.5, -0.18),
            ncol=4,
            frameon=False
        )

        plt.tight_layout()
        plt.savefig(
            os.path.join(args.results_dir, f"learning_curve_{dataset_name}.png"),
            bbox_inches="tight"
        )
        plt.show()

# ==========================================
# 6) ARGS
# ==========================================

class Args:
    seed = 1
    csv_path = "Extract Interface.csv"
    label_col = "refactored"
    results_dir = "results"

    # Active Learning rounds (upper limit; early stopping may end sooner)
    n_round = 35

    # Percent-based labeling (used for AL strategies only; Full ignores these)
    init_pct = 0.05
    query_pct = 0.05
    min_init = 20
    min_query = 20

    # Network hyperparameters
    hidden_dim = 128
    dropout_p = 0.5

    # Training hyperparameters
    n_epochs = 15
    lr = 0.001
    batch_size = 64

    # Dropout-based
    n_drop = 10

    # Adversarial
    bim_eps = 0.05
    bim_iter = 20
    deepfool_iter = 50

    # Convergence / early stopping
    metric_name = "f1"             # "f1" or "accuracy"
    min_delta = 0.001
    patience = 20
    min_rounds_before_stop = 2
    max_label_pct = 50.0

    # Under-sampling controls
    use_undersampling = True
    undersampling_stage = "after_split"   # "before_split" or "after_split"

    strategies = [
        "Full",                 # <-- baseline
        #"RandomSampling",
        #"EntropySampling",
        #"LeastConfidence",
        #"MarginSampling",
        #"KMeansSampling",
        ## "KCenterGreedy",
        #"BALDDropout",
        #"EntropySamplingDropout",
        #"LeastConfidenceDropout",
        #"MarginSamplingDropout",
        #"AdversarialBIM",
        #"AdversarialDeepFool",
    ]

# Example:
# args = Args()
# run_experiment(args)


In [ ]:
# ==========================================
# COLAB CELL: Mount Drive + Run ALL datasets into ONE timestamped run folder
# ==========================================

#from google.colab import drive
import os
import glob
from datetime import datetime

# Mount Drive (already mounted is fine)
#drive.mount('/content/drive', force_remount=False)

DATA_DIR = "datasets" # "/content/drive/MyDrive/AL/datasets"
BASE_RESULTS_DIR = "results" # "/content/drive/MyDrive/AL/results"

# ---- Create timestamped run folder ----
run_id = datetime.now().strftime("run_%Y-%m-%d_%H-%M-%S")
RUN_RESULTS_DIR = os.path.join(BASE_RESULTS_DIR, run_id)
os.makedirs(RUN_RESULTS_DIR, exist_ok=True)

print("📁 Results for this run will be saved to:")
print(RUN_RESULTS_DIR)

# Collect datasets
csv_files = sorted(glob.glob(os.path.join(DATA_DIR, "*.csv")))
print(f"\nFound {len(csv_files)} CSV files")

# ---- Loop over datasets ----
for csv_path in csv_files:
    dataset_name = os.path.splitext(os.path.basename(csv_path))[0]

    print("\n" + "=" * 90)
    print("RUNNING DATASET:", dataset_name)
    print("=" * 90)

    # Fresh Args per dataset
    args = Args()
    args.csv_path = csv_path
    args.results_dir = RUN_RESULTS_DIR   # <-- SAME run folder for all datasets

    run_experiment(args)

print("\n✅ All datasets finished.")
print("📁 All results saved under:", RUN_RESULTS_DIR)
